In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount_retraite.csv', index_col=0)
df.sample(3)

,pays,note,titre,commentaire,Commentaire_filtre,titre_filtre
34201,FR,1,SAV Cdiscount catastrophique,ll y a 4 joursSAV Cdiscount catastrophiqueune ...,"['jourssav', 'cdiscount', 'catastrophiqueune',...","['sav', 'cdiscount', 'catastrophique']"
60446,FR,1,Cdiscount complice d'arnaque,ll y a 5 joursCdiscount complice d'arnaqueJ'ai...,"['jourscdiscount', 'complice', 'arnaquej', 'co...","['cdiscount', 'complice', 'arnaque']"
27808,FR,1,Je me suis fait arnaquer 1400 balles en…,ll y a 5 joursJe me suis fait arnaquer 1400 ba...,"['joursje', 'fait', 'arnaquer', 'balles', 'fai...","['fait', 'arnaquer', 'balles']"


In [3]:
# séparation de la variable cible et des variables explicatives
X = df['Commentaire_filtre']
y = df['note']
print(X.shape)
print(y.shape)

(124120,)
(124120,)


In [4]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [5]:
# conversion des chaines de caratères en tokens numériques

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [13]:
# application du modèle de gradientBoosting sur les tokens numériques et calcul des prédictions
from sklearn.svm import SVC
from sklearn.metrics import classification_report

clf = SVC(C=0.1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# calcul des métriques des résultats du modèle
print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['données prédites'])
display(cm)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14899
           2       1.00      1.00      1.00      2477
           4       1.00      1.00      1.00      1223
           5       1.00      1.00      1.00      6225

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14899,0,0,0
2,0,2477,0,0
4,0,0,1223,0
5,0,0,0,6225


les prédictions sur le test remontent les notes suivantes :  [5 5 5 5]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 5 
 Bel article mais malheureusement trop petit  - la note est de : 5 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 5 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 5


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [8]:
clf2 = SVC(C=10)
clf2.fit(X_train, y_train)

y_pred2 = clf2.predict(X_test)

print(clf2.score(X_test, y_test), end='\n\n')

cr2 = classification_report(y_test, y_pred2)
print(cr2)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm2 = pd.crosstab(y_test, y_pred2, rownames=['données réelles'], colnames=['données prédites'])
display(cm2)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf2.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14899
           2       1.00      1.00      1.00      2477
           4       1.00      1.00      1.00      1223
           5       1.00      1.00      1.00      6225

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14899,0,0,0
2,0,2477,0,0
4,0,0,1223,0
5,0,0,0,6225


les prédictions sur le test remontent les notes suivantes :  [5 5 5 5]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 5 
 Bel article mais malheureusement trop petit  - la note est de : 5 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 5 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 5


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [9]:
clf3 = SVC(C=50)
clf3.fit(X_train, y_train)

y_pred3 = clf3.predict(X_test)

print(clf2.score(X_test, y_test), end='\n\n')

cr3 = classification_report(y_test, y_pred3)
print(cr3)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm3 = pd.crosstab(y_test, y_pred3, rownames=['données réelles'], colnames=['données prédites'])
display(cm3)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf3.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14899
           2       1.00      1.00      1.00      2477
           4       1.00      1.00      1.00      1223
           5       1.00      1.00      1.00      6225

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14899,0,0,0
2,0,2477,0,0
4,0,0,1223,0
5,0,0,0,6225


les prédictions sur le test remontent les notes suivantes :  [5 5 5 5]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 5 
 Bel article mais malheureusement trop petit  - la note est de : 5 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 5 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 5


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [10]:
clf4 = SVC(C=100)
clf4.fit(X_train, y_train)

y_pred4 = clf4.predict(X_test)

print(clf4.score(X_test, y_test), end='\n\n')

cr4 = classification_report(y_test, y_pred4)
print(cr4)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm4 = pd.crosstab(y_test, y_pred4, rownames=['données réelles'], colnames=['données prédites'])
display(cm4)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf4.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14899
           2       1.00      1.00      1.00      2477
           4       1.00      1.00      1.00      1223
           5       1.00      1.00      1.00      6225

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14899,0,0,0
2,0,2477,0,0
4,0,0,1223,0
5,0,0,0,6225


les prédictions sur le test remontent les notes suivantes :  [5 5 5 5]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 5 
 Bel article mais malheureusement trop petit  - la note est de : 5 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 5 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 5


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [11]:
clf5 = SVC(C=200)
clf5.fit(X_train, y_train)

y_pred5 = clf5.predict(X_test)

print(clf5.score(X_test, y_test), end='\n\n')

cr5 = classification_report(y_test, y_pred5)
print(cr5)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm5 = pd.crosstab(y_test, y_pred5, rownames=['données réelles'], colnames=['données prédites'])
display(cm5)

# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites
new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict_test = clf5.predict(new_comments_token)
print('les prédictions sur le test remontent les notes suivantes : ', y_predict_test, ' Détail ci-dessous :\n', new_comments[0], ' - la note est de :', y_predict_test[0], '\n', new_comments[1], ' - la note est de :',  y_predict_test[1], '\n', new_comments[2], ' - la note est de :',  y_predict_test[2], '\n', new_comments[3], ' - la note est de :', y_predict_test[3])

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1.0

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     14899
           2       1.00      1.00      1.00      2477
           4       1.00      1.00      1.00      1223
           5       1.00      1.00      1.00      6225

    accuracy                           1.00     24824
   macro avg       1.00      1.00      1.00     24824
weighted avg       1.00      1.00      1.00     24824



données prédites,1,2,4,5
données réelles,,,,
1,14899,0,0,0
2,0,2477,0,0
4,0,0,1223,0
5,0,0,0,6225


les prédictions sur le test remontent les notes suivantes :  [5 5 5 5]  Détail ci-dessous :
 Satisfait et delai de livraison au top  - la note est de : 5 
 Bel article mais malheureusement trop petit  - la note est de : 5 
 Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés  - la note est de : 5 
 La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article  - la note est de : 5


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


###### le SVC ne fonctionne pas non plus. overfeatting